In [ ]:
version: '3'
services:
  web:
    build: .
    ports:
      - 80:80
    depends_on:
      - database
      - cache

  database:
    image: postgres:latest
    environment:
      - POSTGRES_USER=myuser
      - POSTGRES_PASSWORD=mypassword

  cache:
    image: redis:latest



    import docker
import psutil

def scale_containers(container_name, threshold, max_replicas):
    client = docker.from_env()

    while True:
        cpu_percent = psutil.cpu_percent(interval=1)

        if cpu_percent >= threshold:
            # Get the current number of replicas
            container = client.containers.get(container_name)
            current_replicas = container.attrs['Spec']['Mode']['Replicated']['Replicas']

            # Scale up by 1 if not exceeding the maximum number of replicas
            if current_replicas < max_replicas:
                new_replicas = current_replicas + 1
                print(f"Scaling up to {new_replicas} replicas.")
                container.update(replicas=new_replicas)
        else:
            # Get the current number of replicas
            container = client.containers.get(container_name)
            current_replicas = container.attrs['Spec']['Mode']['Replicated']['Replicas']

            # Scale down by 1 if more than 1 replica
            if current_replicas > 1:
                new_replicas = current_replicas - 1
                print(f"Scaling down to {new_replicas} replicas.")
                container.update(replicas=new_replicas)

# Example usage
container_name = "my_container"  # Replace with your container name
threshold = 70  # CPU usage threshold in percentage
max_replicas = 5  # Maximum number of replicas

scale_containers(container_name, threshold, max_replicas)


#!/bin/bash

# Variables
registry_url="https://your-registry.com"
image_name="your-image"
container_name="your-container"

# Authenticate with the registry
docker login $registry_url

# Pull the latest version of the image
docker pull $registry_url/$image_name

# Run a container based on the image
docker run -d --name $container_name $registry_url/$image_name



In [ ]:
from airflow import DAG
from airflow.operators.bash import BashOperator
from datetime import datetime

# Define the DAG
dag = DAG(
    'shell_command_dag',
    start_date=datetime(2022, 1, 1),
    schedule_interval='@daily',
    catchup=False
)

# Define the task
execute_shell_command = BashOperator(
    task_id='execute_shell_command',
    bash_command='your_shell_command_here',
    dag=dag
)



from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime

def process_input(input_value):
    # Process the input value
    print(f"Processing input: {input_value}")

# Define the DAG
dag = DAG(
    'dynamic_tasks_dag',
    start_date=datetime(2022, 1, 1),
    schedule_interval='@daily',
    catchup=False
)

# Define the list of inputs
input_list = ['input1', 'input2', 'input3']

# Generate tasks dynamically using PythonOperator
tasks = []
for input_value in input_list:
    task = PythonOperator(
        task_id=f'process_input_{input_value}',
        python_callable=process_input,
        op_kwargs={'input_value': input_value},
        dag=dag
    )
    tasks.append(task)

# Set task dependencies
for i in range(1, len(tasks)):
    tasks[i] >> tasks[i-1]


from airflow import DAG
from airflow.operators.dagrun import TriggerDagRunOperator
from datetime import datetime

# Define the DAG
dag = DAG(
    'complex_dependency_dag',
    start_date=datetime(2022, 1, 1),
    schedule_interval=None,
    catchup=False
)

# Define Task A
task_a = TriggerDagRunOperator(
    task_id='task_a',
    trigger_dag_id='my_dag_id',
    execution_date='{{ execution_date }}',
    dag=dag
)

# Define Task B
task_b = TriggerDagRunOperator(
    task_id='task_b',
    trigger_dag_id='my_dag_id',
    execution_date='{{ execution_date }}',
    dag=dag
)

# Set the task dependency
task_a >> task_b


In [ ]:
sqoop import \
--connect jdbc:oracle:thin:@//hostname:port/service \
--username your_username \
--password your_password \
--table your_table \
--columns "column1,column2,column3" \
--target-dir /path/to/hdfs/directory



sqoop import \
--connect jdbc:mysql://hostname:port/database \
--username your_username \
--password your_password \
--table your_table \
--incremental append \
--check-column your_timestamp_column \
--last-value '2022-07-01 00:00:00' \
--merge-key your_primary_key \
--target-dir /path/to/hdfs/directory



sqoop export \
--connect "jdbc:sqlserver://hostname:port;databaseName=your_database" \
--username your_username \
--password your_password \
--table your_table \
--export-dir /path/to/hdfs/directory \
--input-fields-terminated-by ',' \
--input-lines-terminated-by '\n'
